In [20]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
files = [
    "data/01_budgeting_and_tracking.md",
    "data/02_saving_and_investing.md",
    "data/03_debt_and_spending.md"
]
docs = []
for path in files:
    loader = UnstructuredMarkdownLoader(path, mode="single", strategy="fast")
    docs.extend(loader.load())
print(f"Number of chunks: {len(docs)}")
print(docs[0])  # Inspect your first chunk

Number of chunks: 3
page_content='Best Practices for Budgeting and Tracking

1. Create a Detailed Budget

A budget is the cornerstone of good financial management. It's a plan for how you will spend your money each month.

Track Your Income: Know exactly how much money you have coming in from all sources (salary, side hustles, etc.).

Track Your Expenses: Monitor where your money is going. Categorize your spending into areas like housing, transportation, food, and entertainment.

Use Budgeting Tools: Leverage apps like Mint or YNAB, or a simple spreadsheet, to make tracking easier.

2. The 50/30/20 Rule

A popular budgeting guideline is the 50/30/20 rule: * 50% for Needs: Essentials like rent/mortgage, utilities, groceries, and transportation. * 30% for Wants: Non-essential spending like dining out, hobbies, and entertainment. * 20% for Savings and Debt Repayment: Building your savings, investing, and paying off debt.

3. Review and Adjust Regularly

Your financial situation is not sta

In [24]:
!nomic login


                        Authenticate with the Nomic API                         
                        https://atlas.nomic.ai/cli-login                        
  Click the above link to retrieve your access token and then run `nomic login  
                                    [token]`                                    


In [1]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")

In [2]:
from langchain_chroma import Chroma
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="chroma_db_dir"
)


In [ ]:
#only in addng documents once
#vector_store.add_documents(docs)

In [3]:
question = "What are major topics for this class?"
docs = vector_store.similarity_search(question,k=3)
len(docs)

3

In [4]:
import os
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant", api_key=os.getenv("GROQ_API_KEY"))

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Je suis amoureux de la programmation.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 55, 'total_tokens': 66, 'completion_time': 0.021044738, 'prompt_time': 0.003315862, 'queue_time': 0.094194725, 'total_time': 0.0243606}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--65d2d39c-d09c-4c7d-9e85-3ada1a1197e6-0', usage_metadata={'input_tokens': 55, 'output_tokens': 11, 'total_tokens': 66})

In [6]:
from langchain_core.prompts import ChatPromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum. Keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", template),
])

In [7]:
# pip install -U langchain-core langchain langchain-community

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Your existing retriever from the vector store
retriever = vector_store.as_retriever()  # e.g., search_kwargs={"k": 4}

# Your existing prompt
prompt = QA_CHAIN_PROMPT  # ChatPromptTemplate with {context} and {question} slots

# Build the chain: retrieve -> format -> LLM -> parse
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
# Invoke
question = "What is the best practice for saving money ?"
result_text = rag_chain.invoke(question)
print(result_text)

The best practice for saving money is to "Pay Yourself First" by setting up automatic transfers from your checking account to your savings and investment accounts each payday. This ensures you save before you have a chance to spend. Thanks for asking!
